<h2>--- Day 13: Knights of the Dinner Table ---</h2>

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/oddrationale/AdventOfCode2015FSharp/master?urlpath=lab%2Ftree%2FDay13.ipynb)

<p>In years past, the holiday feast with your family hasn't gone so well.  Not everyone gets along!  This year, you resolve, will be different.  You're going to find the <em>optimal seating arrangement</em> and avoid all those awkward conversations.</p>
<p>You start by writing up a list of everyone invited and the amount their happiness would increase or decrease if they were to find themselves sitting next to each other person.  You have a circular table that will be just big enough to fit everyone comfortably, and so each person will have exactly two neighbors.</p>
<p>For example, suppose you have only four attendees planned, and you <span title="Finding a method to calculate happiness units is left as an exercise for the reader.">calculate</span> their potential happiness as follows:</p>
<pre><code>Alice would gain 54 happiness units by sitting next to Bob.
Alice would lose 79 happiness units by sitting next to Carol.
Alice would lose 2 happiness units by sitting next to David.
Bob would gain 83 happiness units by sitting next to Alice.
Bob would lose 7 happiness units by sitting next to Carol.
Bob would lose 63 happiness units by sitting next to David.
Carol would lose 62 happiness units by sitting next to Alice.
Carol would gain 60 happiness units by sitting next to Bob.
Carol would gain 55 happiness units by sitting next to David.
David would gain 46 happiness units by sitting next to Alice.
David would lose 7 happiness units by sitting next to Bob.
David would gain 41 happiness units by sitting next to Carol.
</code></pre>
<p>Then, if you seat Alice next to David, Alice would lose <code>2</code> happiness units (because David talks so much), but David would gain <code>46</code> happiness units (because Alice is such a good listener), for a total change of <code>44</code>.</p>
<p>If you continue around the table, you could then seat Bob next to Alice (Bob gains <code>83</code>, Alice gains <code>54</code>).  Finally, seat Carol, who sits next to Bob (Carol gains <code>60</code>, Bob loses <code>7</code>) and David (Carol gains <code>55</code>, David gains <code>41</code>).  The arrangement looks like this:</p>
<pre><code>     +41 +46
+55   David    -2
Carol       Alice
+60    Bob    +54
     -7  +83
</code></pre>
<p>After trying every other seating arrangement in this hypothetical scenario, you find that this one is the most optimal, with a total change in happiness of <code>330</code>.</p>
<p>What is the <em>total change in happiness</em> for the optimal seating arrangement of the actual guest list?</p>

In [ ]:
open System.Collections.Generic

In [ ]:
let input = File.ReadAllLines @"input/13.txt"

In [ ]:
let parse (line: string) = 
    let split = line.Split(" ")
    let p1 = split.[0]
    let p2 = split.[10].Replace(".", String.Empty)
    let gainOrLose = 
        match split.[2] with
        | "gain" -> 1
        | "lose" -> -1
        | _ -> 0
    let happiness = split.[3] |> int |> (*) gainOrLose
    p1, p2, happiness

Create a dictionary of dictionaries. First set of keys would be the initial person, the second key would be the person the are sitting next to. E.g., using the example above, `happinessMap.["Alice"].["Bob"]` equals `54`.

In [ ]:
let createHappinessMap (input: string[]) = 
    input
    |> Seq.map parse
    |> Seq.groupBy (fun (p1, _, _) -> p1)
    |> Seq.map (fun (grp, values) -> grp, dict [for (_, p2, h) in values -> p2, h])
    |> dict

In [ ]:
let happinessMap = createHappinessMap input

Got the `permute` function [from StackOverflow](https://stackoverflow.com/a/3129136).

In [ ]:
let rec distribute e = function
  | [] -> [[e]]
  | x::xs' as xs -> (e::xs)::[for xs in distribute e xs' -> x::xs]

let rec permute = function
  | [] -> [[]]
  | e::xs -> List.collect (distribute e) (permute xs)

In [ ]:
#!time
happinessMap.Keys
|> List.ofSeq
|> permute
|> Seq.map (fun arr -> 
    arr
    |> Seq.windowed 2
    |> Seq.append [|[|arr |> Seq.last; arr |> Seq.head|]|]
    |> Seq.map (fun pair -> happinessMap.[pair.[0]].[pair.[1]] + happinessMap.[pair.[1]].[pair.[0]])
    |> Seq.sum
)
|> Seq.max

733

Wall time: 125.3745ms

<h2 id="part2">--- Part Two ---</h2>

<p>In all the commotion, you realize that you forgot to seat yourself.  At this point, you're pretty apathetic toward the whole thing, and your happiness wouldn't really go up or down regardless of who you sit next to.  You assume everyone else would be just as ambivalent about sitting next to you, too.</p>
<p>So, add yourself to the list, and give all happiness relationships that involve you a score of <code>0</code>.</p>
<p>What is the <em>total change in happiness</em> for the optimal seating arrangement that actually includes yourself?</p>

In [ ]:
let findHappiness (happinessMap: IDictionary<string,IDictionary<string, int>>) pair = 
    let p1 = pair |> Seq.head
    let p2 = pair |> Seq.last
    if happinessMap.ContainsKey p1 then
        if happinessMap.[p1].ContainsKey p2 then
            happinessMap.[p1].[p2]
        else 0
    else 0

In [ ]:
#!time
happinessMap.Keys
|> List.ofSeq
|> List.append ["Me"]
|> permute
|> Seq.map (fun arr ->
    arr
    |> Seq.windowed 2
    |> Seq.append [|[|arr |> Seq.last; arr |> Seq.head|]|]
    |> Seq.map (fun pair -> (findHappiness happinessMap pair) + (findHappiness happinessMap (Seq.rev pair)))
    |> Seq.sum
)
|> Seq.max

725

Wall time: 3260.1303ms

[Prev](Day12.ipynb) | [Next](Day14.ipynb)